# Instrukcja - Transformacja Hougha

### Cel:
- zapoznanie z transformacją Hougha dla pojedynczego punktu,
- kilku punktów, prostych figur
- wykorzystanie transformacji Hougha do detekcji linii prostych na rzeczywistym obrazie
- transformacja Hougha w przestrzeni ab

### Transformacja Hough'a

Transformacja Hougha dla prostych jest metodą detekcji współliniowych punktów. Każda prosta może być jednoznacznie przedstawiona za pomocą dwóch parametrów. Przestrzeń tych parametrów to przestrzeń Hougha. Najczęściej wykorzystywanymi parametrami w tej metodzie są współczynniki ρ,θ

opisujące równanie prostej w postaci normalnej:

ρ=x⋅cos(θ)+y⋅sin(θ)

gdzie: ρ - promień wodzący, θ - kąt pomiędzy ρ a osią OX.

Własności transformacji Hougha:
- prostej w przestrzeni kartezjańskiej odpowiada punkt w przestrzeni Hougha
- pękowi prostych przechdzących przez punkt w przestrzeni kartezjańskiej odpowiada krzywa sinusoidalna w przestrzeni Hougha
- punkty leżące na tej samej prostej (w przestrzeni kartezjańskiej) korespondują z sinusoidami przechodzącymi przez wspólny punkt w przestrzeni Hougha.

Metoda wyliczania transformacji Hougha składa się z następujących kroków:
- przez każdy badany (różny od zera) punkt obrazu prowadzony jest pęk prostych, przechodzących przez ten punkt
- każda z tych prostych transformowana jest do przestrzeni Hougha i tworzy tam punkt o współrzędnych ρ,θ
- w ten sposób, każdy punkt obrazu pierwotnego (pęk prostych) jest odwzorowany w sinusoidalną krzywą w przestrzeni Hougha

Przestrzeń Hougha jest przestrzenią akumulacyjną tzn. punkty sinusoidalnych krzywych, wygenerowanych dla punktów obrazu pierwotnego dodają się w miejscach, w których krzywe te przecinają się. Powstałe w ten sposób (w przestrzeni Hougha) maksima odpowiadają zbiorom punktów, należących do jednej prostej. Współrzędne ρ,θ
tego maksimum jednoznacznie określają położenie prostej na obrazie pierwotnym.

### Transformacja Hougha dla małej liczby punktów.
   1. Uruchom poniższy kod. W tablicy `im` wskaż jeden punkt, dla którego ma zostać obliczona transformata.

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from skimage.transform import hough_line, hough_line_peaks
import os

if not os.path.exists("kwadraty.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/11_Hough/kwadraty.png --no-check-certificate
if not os.path.exists("lab112.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/11_Hough/lab112.png --no-check-certificate
if not os.path.exists("dom.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/11_Hough/dom.png --no-check-certificate

im = np.zeros((64,64), dtype=np.uint8)

im[32, 32] = 1

fig, ax = plt.subplots()
fig.set_size_inches(4, 4)
ax.imshow(im, 'gray')
ax.axis('off')


3. Wykonaj transformację Hougha obazu im. Wykorzystaj funkcję *hough_line* z modułu _skimage.transform_. Funkcja zwraca: macierz H (przestrzeń Hougha) oraz dwa wektory theta i rho dla kolejnych 
4. Wyświetl przestrzeń Hougha za pomocą funkcji _plt.imshow_ (można też wykorzystać poniższą funkcję *show_hough*). Jak "wygląda" pojedynczy punkt w przestrzeni Hougha?

In [ ]:
def show_hough(h, image):
    # Generating figure 1
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    ax = axes.ravel()

    ax[0].imshow(image, 'gray')
    ax[0].set_title('Input image')
    ax[0].set_axis_off()

    ax[1].imshow(h, 'gray')
    ax[1].set_title('Hough transform')
    ax[1].set_xlabel('Angles (degrees)')
    ax[1].set_ylabel('Distance (pixels)')
    ax[1].axis('image')
    
    plt.tight_layout()
    plt.show()    

In [ ]:
hough = hough_line(im)
show_hough(hough[0], im)

Jak funkcja sinusoidalna

In [ ]:
im1 = np.zeros((64,64), dtype=np.uint8)
im1[32, 32] = 1
im1[20, 20] = 1
im1[44, 10] = 1
hough = hough_line(im1)
show_hough(hough[0], im1)

5. Powtórz punkty 1-4, ale tym razem klinkij dwa punkty. Jak zmienia się przestrzeń Hougha?
6. Powtórz punkty 1-4, ale tym razem kliknij kilka punktów starając się aby były współliniowe. Zaobserwuj zmiany w przestrzeni Hougha
7. Poeksperymentuj z różnymi układami punktów

### Transformata Hougha dla pojedynczego obiektu

W tym podpunkcie pokazane zostanie praktycznie wykorzystanie transformaty Hougha - do detekcji prostych na sztucznym rysunku.

   1. Wczytaj obraz "kwadraty.png". Wyświetl go.
   2. Wykonaj detekcję krawędzi jedną z metod gradientowych. Ważne aby obraz krawędzi był jak najlepszej jakości - co oznacza cienkie (nawet niekoniecznie ciągłe) krawędzie - dla tego przypadku nie powinno być trudne do uzyskania. Wyświetl obraz po detekcji krawędzi.
   3. Wykonaj transformatę Hougha obrazu krawędziowego. Wykorzystaj funkcję *hough\_line*.
   4. Wyświetl macierz H. Czy widoczna jest taka liczba maksimów jakiej się spodziewamy?

In [ ]:
kwadrat = cv2.imread('kwadraty.png',cv2.IMREAD_GRAYSCALE)
kwa_can=cv2.Canny(kwadrat,5,10,None,3,1)
plt.imshow(kwa_can, 'gray')
plt.axis('off')
plt.show()

In [ ]:
hough_kwadrat = hough_line(kwa_can)
show_hough(hough_kwadrat[0], kwa_can)

Tak widoczna jest właściwa liczba maksimów, ale widać to dopiero po narysowaniu okręgów.

 5. W module skimage.transform dostępna jest funkcja do automatycznej analizy przestrzeni Hougha - wyszukiwania maksimów - *hough\_line\_peaks*. Jako parametry przyjmuje ona wyniki funkcji *hough\_line* (macierz H, theta i rho). Dodatkowo można podać próg powyżej którego punkt uznawany jest za maksimum (_threshold_ - domyslnie jest to połowa maksimum w przestrzeni H) oraz liczbę poszukiwanych maksimów (*num_peaks*). Funkcja zwraca współrzędne maksimów. Wykorzystaj funkcję *hough\_line\_peaks* do znalezienia maksimów odpowiadających krawędziom kwadratów.
 6. Wyświetl macierz H używając konstrukcji:

In [ ]:
h, angles, distances  = hough_line_peaks(hough_kwadrat[0],hough_kwadrat[1],hough_kwadrat[2])
print(angles, distances)

In [ ]:
import math
angles = (angles*(180/math.pi))
angles +=90
distances+=363
circle = []
for i in range(len(angles)):
    circle.append(plt.Circle((angles[i], distances[i]), 10, color='r', fill=False))

In [ ]:
fig,ax = plt.subplots(1,figsize=(20,15))
ax.set_aspect('equal')
ax.imshow(hough_kwadrat[0], 'gray')
for j in range(len(circle)):
    ax.add_patch(circle[j])


Taki zapis pozwoli na dołożenie annotacji (okręgów) w miejscach znalezionych maksimów. Wyrysowanie okręgu w punkcie x, y (o rozmiarze 10, w czerwonym kolorze, bez wypełnienia środka) realizuje wywołanie: 

**circle = plt.Circle((x, y), 10, color='r', fill=False)**

natomiast dołożenie takiego okręgu do obrazu to:

**ax.add_patch(circle)**

Zaznacz maksima na obrazie wykorzystując rezultat funkcji *hough\_line\_peaks* biorąc pod uwagę, że zwraca ona kąty w radianach z przedziału od -pi/2 do pi/2, a rho z przedziału od -r/2 do r/2 gdzie r to pionowy rozmiar przestrzeni Hougha. 

7. Istnieje też możliwość przeprowadzenia transformacji Hougha z użyciem biblioteki OpenCV. W bibliotece znajdują się dwie wersje funkcji wyszukującej linie proste - 'klasyczna' - _HoughLines_ oraz probabilistyczna _HoughLinesP_. Zadna z nich nie zwraca przestrzeni Hougha. Wynikiem działania pierwszej jest lista parametrów prostych (krotki zawierające rho, theta). Druga zwraca krotki 4-ro elementowe ze współrzędnymi końców odcinków wykorzystanych do wylicznia parametrów (czyli znalezienia prostej). 
8. Wyznacz linie obecne na obrazie za pomocą funkcji _HoughLines_. Wykryte linie wyrysuj na obrazie początkowym (UWAGA: wczytanym bez konwersji na graylevel). Do wyświetlania linii wykorzystaj przykładowy kod:

In [ ]:
I_kwadraty = cv2.imread('kwadraty.png')
edges = cv2.Canny(I_kwadraty,70,150,apertureSize = 3)
lines = cv2.HoughLines(edges,1,math.pi/180,65)

In [ ]:
for line in lines:
    rho,theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))

    cv2.line(I_kwadraty,(x1,y1),(x2,y2),(0,0,255),2)

In [ ]:
plt.imshow(I_kwadraty, 'gray')
plt.axis('off')
plt.show()

9. Wyznacz odcinki obecne na obrazie za pomocą funkcji _HoughLinesP_. Wykryte odcinki wyrysuj na obrazie początkowym (UWAGA: wczytanym bez konwersji na graylevel). 

In [ ]:
I_kwadraty_2 = cv2.imread('kwadraty.png')
lines_2 = cv2.HoughLinesP(edges,1,np.pi/180,40,minLineLength=20,maxLineGap=10)
for line in lines_2:
    x1,y1,x2,y2 = line[0]
    cv2.line(I_kwadraty_2,(x1,y1),(x2,y2),(0,255,0),2)

In [ ]:
plt.imshow(I_kwadraty_2, 'gray')
plt.axis('off')
plt.show()

### Transformata Hougha dla obrazu rzeczywistego.

Bazując na kodzie stworzonym w punkcie B wyszukamy linie na obrazie rzeczywistym.
   1. Wczytaj obraz "lab112.png". Wyświetl go.
   2. Wykorzystując wszystkie poznane techniki przetwarzania obrazów (filtracja, przekształcenia morfologiczne, binaryzację, detekcję krawędzi) wyodrębnij krawędzie samych kwadratów - tak aby były jak najlepszej jakości (cienkie) - jednocześnie eliminując z obrazu zakłócenia.
   3. Wykorzystaj funkcje *hough_line* i *hough_line_peaks* do detekcji linii na obrazie, a następnie np. wykorzystując kod z punktu 8 poprzedniego ustępu wyrysuj na oryginalnym obrazie znalezione linie.

In [ ]:
I_lab_rgb = cv2.imread('lab112.png')
I_lab = cv2.imread('lab112.png',cv2.IMREAD_GRAYSCALE)
I_lab=np.where(I_lab>40,0,255)
I_lab[:,500:]=0
I_lab = I_lab.astype('uint8')
plt.imshow(I_lab,'gray')
plt.axis('off')
plt.show()
X,Y = I_lab.shape
puste = np.zeros((X,Y,3))
puste[:,:,0] = I_lab
puste[:,:,1] = I_lab
puste[:,:,2] = I_lab
puste = puste.astype('uint8')

edges = cv2.Canny(puste,50,150,apertureSize = 3)

In [ ]:
lab112_H, lab112_theta, lab112_rho = hough_line(edges)
lab112_H_2, lab112_theta_2, lab112_rho_2 = hough_line_peaks(lab112_H, lab112_theta, lab112_rho)
theta2 = lab112_theta_2
rho2 = lab112_rho_2
wynik = cv2.cvtColor(I_lab, cv2.COLOR_GRAY2BGR)

for i in range(0,len(theta2)):
    rho = rho2[i]
    theta = theta2[i]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))
        
    pt1 = (x1, y1)
    pt2 = (x2, y2)
    cv2.line(wynik, pt1, pt2, (255,0,0), 2, cv2.LINE_AA)

plt.imshow(wynik)

4. Wczytaj obraz "dom.png". Wypróbuj działanie transformacji Hougha na tym obrazie z wykorzystaniem funkcji _cv2.HoughLinesP_  (oczywiście po odpowiednich przekształceniach). Postaraj się tak przygotować obraz z krawędziami i dobrać parametry aby wyrysować na oryginalnym obrazie odcinki obejmujące zarysy domu. Weź pod uwage dodatkowe parametry funkcji, takie jak:   minLineLength, maxLineGap.

In [ ]:
I_dom_rgb = cv2.imread('dom.png')
I_dom = cv2.imread('dom.png',cv2.IMREAD_GRAYSCALE)

plt.imshow(I_dom, 'gray')
plt.axis('off')
plt.show()

In [ ]:
I_dom_rgb = cv2.imread('dom.png')
I_dom = cv2.imread('dom.png',cv2.IMREAD_GRAYSCALE)
I_dom_med=cv2.GaussianBlur(I_dom, (15, 15), 0)
I_dom_med=cv2.GaussianBlur(I_dom_med, (7, 7), 0)

I_dom_can=cv2.Canny(I_dom_med,10,30,None,3,1)
lines_3 = cv2.HoughLinesP(I_dom_can,1,np.pi/180,40,minLineLength=40,maxLineGap=5)
for line in lines_3:
    x1,y1,x2,y2 = line[0]
    cv2.line(I_dom_rgb,(x1,y1),(x2,y2),(255,0,0),2)
    
    
plt.imshow(I_dom_rgb)
plt.axis('off')
plt.show()